In [1]:
import numpy as np
import pandas as pd

In [2]:
train_df=pd.read_csv('../dataset/train.csv')

In [3]:
train_df.head()

,image_id,breed
0,a8b3ad1dde,nerodia-erythrogaster
1,8b492b973d,pantherophis-vulpinus
2,929b99ea92,thamnophis-sirtalis
3,bbac7385e2,pantherophis-obsoletus
4,ef776b1488,agkistrodon-contortrix


In [4]:
for i in range(len(train_df['image_id'])):
    train_df['image_id'][i]=train_df['image_id'][i]+'.jpg'

In [5]:
len(train_df['image_id'])

5508

In [6]:
train_df.head()

,image_id,breed
0,a8b3ad1dde.jpg,nerodia-erythrogaster
1,8b492b973d.jpg,pantherophis-vulpinus
2,929b99ea92.jpg,thamnophis-sirtalis
3,bbac7385e2.jpg,pantherophis-obsoletus
4,ef776b1488.jpg,agkistrodon-contortrix


In [7]:
TRAIN_SPLIT = 0.75
# the amount of validation data will be a percentage of the
# *training* data
VAL_SPLIT = 0.1

INIT_LR = 1e-4
NUM_EPOCHS = 20
BS = 128
# define the path to the serialized output model after training
MODEL_PATH = "camo_detector.model"

In [8]:
len(train_df)

5508

In [9]:
i = int(len(train_df) * TRAIN_SPLIT)
train_dataframe = train_df[:i]
test_dataframe = train_df[i:]

i = int(len(train_dataframe) * VAL_SPLIT)
val_dataframe = train_dataframe[:i]
train_dataframe = train_dataframe[i:]

totalTrain = len(train_dataframe)
totalVal = len(val_dataframe)
totalTest = len(test_dataframe)

print(len(train_dataframe), len(val_dataframe), len(test_dataframe))

3718 413 1377


In [10]:
from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rotation_range=25,
                                   zoom_range=0.1,
                                   width_shift_range=0.1,
                                   height_shift_range=0.1,
                                   shear_range=0.2,
                                   horizontal_flip=True,
                                   fill_mode="nearest")

test_datagen = ImageDataGenerator()

# define the ImageNet mean subtraction (in RGB order) and set the
# the mean subtraction value for each of the data augmentation
# objects
mean = np.array([123.68, 116.779, 103.939], dtype="float32")
train_datagen.mean = mean
test_datagen.mean = mean

Using TensorFlow backend.


In [11]:
train_generator=train_datagen.flow_from_dataframe(dataframe=train_dataframe,
                                                  directory="../dataset/train/",
                                                  x_col="image_id",
                                                  y_col="breed",
                                                  subset="training",
                                                batch_size=BS,
                                                seed=42,
                                                shuffle=True,
                                                color_mode="rgb",
                                                class_mode="categorical",
                                                target_size=(224, 224))

val_generator=test_datagen.flow_from_dataframe(dataframe=val_dataframe,
                                                directory="../dataset/train/",
                                                x_col="image_id",
                                                y_col="breed",
                                                subset="training",
                                                batch_size=BS,
                                                shuffle=False,
                                                color_mode="rgb",
                                                class_mode="categorical",
                                                target_size=(224, 224))

test_generator=test_datagen.flow_from_dataframe(dataframe=test_dataframe,
                                                directory="../dataset/train/",
                                                x_col="image_id",
                                                y_col="breed",
                                                subset="training",
                                                batch_size=BS,
                                                shuffle=False,
                                                color_mode="rgb",
                                                class_mode="categorical",
                                                target_size=(224, 224))

Found 3718 validated image filenames belonging to 35 classes.
Found 413 validated image filenames belonging to 35 classes.
Found 1377 validated image filenames belonging to 35 classes.


In [17]:
from keras.applications.resnet50 import ResNet50
from keras.models import Model
from keras.layers import Input, Dropout, Dense, Flatten, AveragePooling2D
from tensorflow.keras.optimizers import Adam

In [13]:
baseModel = ResNet50(weights="imagenet", include_top=False, input_tensor=Input(shape=(224, 224, 3)))

/opt/conda/lib/python3.7/site-packages/keras_applications/resnet50.py:265: UserWarning: The output shape of `ResNet50(include_top=False)` has been changed since Keras 2.2.0.
  warnings.warn('The output shape of `ResNet50(include_top=False)` '


In [14]:
# construct the head of the model that will be placed on top of the
# the base model
headModel = baseModel.output
headModel = AveragePooling2D(pool_size=(7, 7))(headModel)
headModel = Flatten(name="flatten")(headModel)
headModel = Dense(256, activation="relu")(headModel)
headModel = Dropout(0.5)(headModel)
headModel = Dense(35, activation="softmax")(headModel)
# place the head FC model on top of the base model (this will become
# the actual model we will train)
model = Model(inputs=baseModel.input, outputs=headModel)
# loop over all layers in the base model and freeze them so they will
# *not* be updated during the training process
for layer in baseModel.layers:
    layer.trainable = False

In [15]:
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 112, 112, 64) 256         conv1[0][0]                      
____________________________________________________________________________________________

In [18]:
opt = Adam(lr=INIT_LR, decay=INIT_LR / NUM_EPOCHS)
model.compile(loss="binary_crossentropy", optimizer=opt, metrics=["accuracy"])
# train the model
print("[INFO] training model...")
H = model.fit_generator(train_generator, steps_per_epoch=totalTrain // BS, validation_data=val_generator,
                        validation_steps=totalVal // BS, epochs=NUM_EPOCHS)

[INFO] training model...
Epoch 1/20


KeyboardInterrupt: 